In [ ]:
import os
import sys

import kodo

# Choose the finite field, the number of symbols (i.e. generation size)
# and the symbol size in bytes
field = kodo.field.binary
symbols = 42
symbol_size = 160

# Create an encoder and two decoders
encoder = kodo.RLNCEncoder(field, symbols, symbol_size)
decoder1 = kodo.RLNCDecoder(field, symbols, symbol_size)
decoder2 = kodo.RLNCDecoder(field, symbols, symbol_size)

# Generate some random data to encode. We create a bytearray of the same
# size as the encoder's block size and assign it to the encoder.
# This bytearray must not go out of scope while the encoder exists!
data_in = bytearray(os.urandom(encoder.block_size()))
encoder.set_symbols_storage(data_in)

# Define the data_out bytearrays where the symbols should be decoded
# These bytearrays must not go out of scope while the encoder exists!
data_out1 = bytearray(decoder1.block_size())
data_out2 = bytearray(decoder1.block_size())
decoder1.set_symbols_storage(data_out1)
decoder2.set_symbols_storage(data_out2)

while not decoder2.is_complete():

    # Encode a packet into the payload buffer
    packet = encoder.produce_payload()

    # Pass that packet to decoder1
    decoder1.consume_payload(packet)

    # Now produce a new recoded packet from the current
    # decoding buffer, and place it into the payload buffer
    packet = decoder1.produce_payload()

    # Pass the recoded packet to decoder2
    decoder2.consume_payload(packet)

# Both decoder1 and decoder2 should now be complete,
# check if the output buffers match the data_in buffer
if data_out1 == data_in and data_out2 == data_in:
    print("Data decoded correctly")
else:
    print("Unexpected failure to decode please file a bug report :)")
    sys.exit(1)